# Comparação dos dados brutos (HNE) e Processados para o primeiro semetre de 2012

In [5]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pylab as pl

In [13]:
# carrega dados brutos da boia

pathname = os.environ['HOME'] + '/Dropbox/pnboia/data/'

rig = pd.read_csv(pathname + 'rio_grande/proc/rig_8_lioc.csv', parse_dates=['date'], index_col='date')
fln = pd.read_csv(pathname + 'florianopolis/proc/fln_8_lioc.csv', parse_dates=['date'], index_col='date')
san = pd.read_csv(pathname + 'santos/proc/san_8_lioc.csv', parse_dates=['date'], index_col='date')

# corrige a declinacao magnetica
rig.dp = rig.dp - 17
fln.dp = fln.dp - 20
san.dp = san.dp - 22

rig.dp.loc[rig.dp < 0] = rig.dp.loc[rig.dp < 0] + 360
fln.dp.loc[fln.dp < 0] = fln.dp.loc[fln.dp < 0] + 360
san.dp.loc[san.dp < 0] = san.dp.loc[san.dp < 0] + 360

In [4]:
# carrega dados processados validados pela MB

pathname = os.environ['HOME'] + '/Dropbox/pnboia/data/'

dd = pd.read_excel(pathname + 'historico_dados_validados.xlsx',
                   sheetname=['METADADOS','santos','santacatarina','riogrande','recife','portoseguro','cabofrio'])

rg = dd['riogrande']
re = dd['recife']
sa = dd['santos']
fl = dd['santacatarina']
ps = dd['portoseguro']
cf = dd['cabofrio']

rg['date'] = [datetime(rg.Year[i],rg.Month[i],rg.Day[i],rg.Hour[i]) for i in range(len(rg))]
fl['date'] = [datetime(fl.Year[i],fl.Month[i],fl.Day[i],fl.Hour[i]) for i in range(len(fl))]
sa['date'] = [datetime(sa.Year[i],sa.Month[i],sa.Day[i],sa.Hour[i]) for i in range(len(sa))]
re['date'] = [datetime(re.Year[i],re.Month[i],re.Day[i],re.Hour[i]) for i in range(len(re))]
ps['date'] = [datetime(ps.Year[i],ps.Month[i],ps.Day[i],ps.Hour[i]) for i in range(len(ps))]
cf['date'] = [datetime(cf.Year[i],cf.Month[i],cf.Day[i],cf.Hour[i]) for i in range(len(cf))]

#consistencia dos dados validados
rg = rg.loc[(rg.wvht < 10) & (rg.wvht > 0) & (rg.dpd < 25)]
fl = fl.loc[(fl.wvht < 10) & (fl.wvht > 0) & (fl.dpd < 25)]
sa = sa.loc[(sa.wvht < 10) & (sa.wvht > 0) & (sa.dpd < 25)]
cf = cf.loc[(cf.wvht < 10) & (cf.wvht > 0) & (cf.dpd < 25)]
ps = ps.loc[(ps.wvht < 10) & (ps.wvht > 0) & (ps.dpd < 25)]
re = re.loc[(re.wvht < 10) & (re.wvht > 0) & (re.dpd < 25)]

rg.index = rg.date
fl.index = fl.date
sa.index = sa.date

In [6]:
# carrega dados do summary

#Saida da Axys
#  0        1            2          3        4        5         6        7            8            9         10     11           12       13      14              15  
#YearJulian Date/Zero Crossings/Ave. Ht./Ave. Per./Max Ht./Sig. Wave/Sig. Per./Peak Per.(Tp)/Peak Per.(READ)/HM0/Mean Theta/Sigma Theta/ H1/10 / T.H1/10	/Mean Per.(Tz)

pathname = os.environ['HOME'] + '/Dropbox/pnboia/data/rio_grande/proc/'

ax = np.loadtxt(pathname + 'rig_summary.txt',skiprows = 1, usecols = (range(2,18)))
ax_data = np.loadtxt(pathname + 'rig_summary.txt',dtype = str, skiprows = 1, usecols = (0,1))

#deixa datas com numeros inteiros
ano_ax = [int(ax_data[i,0][0:4]) for i in range(len(ax_data))]
mes_ax = [int(ax_data[i,0][5:7]) for i in range(len(ax_data))]
dia_ax = [int(ax_data[i,0][8:10]) for i in range(len(ax_data))]
hora_ax = [int(ax_data[i,1][:2]) for i in range(len(ax_data))]
min_ax = [int(ax_data[i,1][3:]) for i in range(len(ax_data))]

datam_ax = []
for i in range(len(ax_data)):
    datam_ax.append(datetime(ano_ax[i],mes_ax[i],dia_ax[i],hora_ax[i],min_ax[i]))

#axys
hm0_ax = ax[:,10]
tp_ax = ax[:,8] #qual periodo usar?
tmed_ax = ax[:,15]
th10_ax = ax[:,14]
dirtp_ax = ax[:,11]
hmax_ax = ax[:,5]
hs_ax = ax[:,6]
h10_ax = ax[:,13]

In [19]:
%matplotlib qt5

pl.figure()
pl.plot(datam_ax, hm0_ax,'.')
pl.plot(rig.index, rig.hm0,'.')

pl.show()

In [18]:
rg[['wvht','dpd','mwd']].to_csv('../out/rig_mb.csv')
fl[['wvht','dpd','mwd']].to_csv('../out/fln_mb.csv')
sa[['wvht','dpd','mwd']].to_csv('../out/san_mb.csv')

In [15]:
%matplotlib qt5

pl.close('all')

pl.figure()
pl.subplot(311)
pl.title('RIG')
pl.plot(rig.hm0,'.')
pl.plot(rg.wvht,'.')
pl.subplot(312)
pl.plot(rig.tp,'.')
pl.plot(rg.dpd,'.')
pl.subplot(313)
pl.plot(rig.dp,'.')
pl.plot(rg.mwd,'.')

pl.figure()
pl.subplot(311)
pl.title('FLN')
pl.plot(fln.hm0,'.')
pl.plot(fl.wvht,'.')
pl.subplot(312)
pl.plot(fln.tp,'.')
pl.plot(fl.dpd,'.')
pl.subplot(313)
pl.plot(fln.dp,'.')
pl.plot(fl.mwd,'.')

pl.figure()
pl.subplot(311)
pl.title('SAN')
pl.plot(san.hm0,'.')
pl.plot(sa.wvht,'.')
pl.subplot(312)
pl.plot(san.tp,'.')
pl.plot(sa.dpd,'.')
pl.subplot(313)
pl.plot(san.dp,'.')
pl.plot(sa.mwd,'.')

pl.show()